<a href="https://colab.research.google.com/github/ychoi-kr/LLM-API/blob/main/upstage/solar_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 솔라 임베딩 API

In [13]:
!pip install openai

In [14]:
from google.colab import userdata
import os

api_key = userdata.get('UPSTAGE_API_KEY')

In [15]:
from openai import OpenAI
client = OpenAI(
    api_key=api_key,
    base_url="https://api.upstage.ai/v1/solar"
)

## 임베딩

In [16]:
response = client.embeddings.create(
    input="왕",
    model="solar-embedding-1-large-passage"
)

king = response.data[0].embedding

In [17]:
king

[0.0102386474609375,
 0.005893707275390625,
 -0.0134735107421875,
 0.0223388671875,
 -0.006168365478515625,
 -0.0030460357666015625,
 -0.016510009765625,
 -0.01409912109375,
 0.0032958984375,
 -0.0008378028869628906,
 0.0103759765625,
 0.01184844970703125,
 -0.00788116455078125,
 -0.022064208984375,
 0.0016984939575195312,
 0.020172119140625,
 -0.0196685791015625,
 0.01030731201171875,
 0.00798797607421875,
 -0.0192718505859375,
 -0.01294708251953125,
 0.00860595703125,
 -0.00777435302734375,
 0.0139312744140625,
 -0.0031070709228515625,
 -0.0041046142578125,
 0.0086669921875,
 0.0038852691650390625,
 0.012786865234375,
 0.00812530517578125,
 0.0138702392578125,
 -0.00823974609375,
 0.00960540771484375,
 -0.0112152099609375,
 -0.007717132568359375,
 -0.0078582763671875,
 -0.01251983642578125,
 0.006229400634765625,
 -0.008514404296875,
 -0.0080413818359375,
 0.0142364501953125,
 0.00820159912109375,
 -0.0267181396484375,
 -0.002376556396484375,
 -0.01194000244140625,
 -0.0135498046875,

In [18]:
len(king)

4096

## 임베딩 함수 정의

In [19]:
def embedding_passage(input):
    response = client.embeddings.create(
        input=input,
        model="solar-embedding-1-large-passage"
    )

    return response.data[0].embedding

def embedding_query(input):
    response = client.embeddings.create(
        input=input,
        model="solar-embedding-1-large-query"
    )

    return response.data[0].embedding

## 코사인 유사도

In [20]:
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

## 비슷한 단어

In [21]:
import pandas as pd

df = pd.DataFrame({'단어': ['공부', '글', '도서', '문서', '영화', '책상', '화장품']})
df['embedding'] = df['단어'].apply(embedding_passage)
df

,단어,embedding
0,공부,"[0.02081298828125, 0.014404296875, -0.01062011..."
1,글,"[0.0311126708984375, -0.0045318603515625, -0.0..."
2,도서,"[0.0014438629150390625, -0.0022449493408203125..."
3,문서,"[0.0208740234375, -0.0011138916015625, -0.0191..."
4,영화,"[0.00628662109375, -0.01092529296875, -0.00377..."
5,책상,"[0.0112152099609375, -0.030242919921875, 0.002..."
6,화장품,"[0.005100250244140625, -0.002788543701171875, ..."


In [22]:
df['cosine_similarity'] = df['embedding'].apply(lambda x: cosine_similarity(x, embedding_query('책')))
df.sort_values(by='cosine_similarity', ascending=False)

,단어,embedding,cosine_similarity
2,도서,"[0.0014438629150390625, -0.0022449493408203125...",0.292631
5,책상,"[0.0112152099609375, -0.030242919921875, 0.002...",0.230436
1,글,"[0.0311126708984375, -0.0045318603515625, -0.0...",0.203212
0,공부,"[0.02081298828125, 0.014404296875, -0.01062011...",0.200380
4,영화,"[0.00628662109375, -0.01092529296875, -0.00377...",0.162114
3,문서,"[0.0208740234375, -0.0011138916015625, -0.0191...",0.162068
6,화장품,"[0.005100250244140625, -0.002788543701171875, ...",0.105274


## 비슷한 속담 찾기

In [23]:
import pandas as pd

proverbs = [
    "가는 말이 고와야 오는 말이 곱다",
    "가는 날이 장날이다",
    "개구리 올챙이 적 생각 못 한다",
    "고래 싸움에 새우 등 터진다",
    "공든 탑이 무너지랴",
    "구슬이 서 말이라도 꿰어야 보배",
    "그림의 떡",
    "김칫국부터 마신다",
    "낫 놓고 기역 자도 모른다",
    "누워서 떡 먹기",
    "등잔 밑이 어둡다",
    "뛰는 놈 위에 나는 놈 있다",
    "말 한마디로 천 냥 빚 갚는다",
    "매도 먼저 맞는 게 낫다",
    "모로 가도 서울만 가면 된다",
    "물이 깊을수록 소리가 없다",
    "바늘 도둑이 소 도둑 된다",
    "배보다 배꼽이 크다",
    "백지장도 맞들면 낫다",
    "보기 좋은 떡이 먹기도 좋다",
    "콩 심은 데 콩 나고 팥 심은 데 팥 난다",
    "티끌 모아 태산",
    "시작이 반이다"
]

df = pd.DataFrame({'속담': proverbs})
df['embedding'] = df['속담'].apply(embedding_passage)
df

,속담,embedding
0,가는 말이 고와야 오는 말이 곱다,"[-0.00739288330078125, 0.009613037109375, -0.0..."
1,가는 날이 장날이다,"[-0.006801605224609375, -0.00921630859375, -0...."
2,개구리 올챙이 적 생각 못 한다,"[-0.00926971435546875, 0.0004954338073730469, ..."
3,고래 싸움에 새우 등 터진다,"[0.0036220550537109375, -0.0070648193359375, -..."
4,공든 탑이 무너지랴,"[-0.01824951171875, -0.017059326171875, -0.017..."
5,구슬이 서 말이라도 꿰어야 보배,"[-0.0124053955078125, 0.0128173828125, -0.0065..."
6,그림의 떡,"[0.0017900466918945312, 0.0024890899658203125,..."
7,김칫국부터 마신다,"[-0.0004630088806152344, -0.01076507568359375,..."
8,낫 놓고 기역 자도 모른다,"[0.01183319091796875, 0.001712799072265625, 0...."
9,누워서 떡 먹기,"[-0.006122589111328125, -0.0021457672119140625..."


In [24]:
# df의 각 속담에 대해 코사인 유사도를 계산하고 내림차순 정렬

df['cosine_similarity'] = df['embedding'].apply(
    lambda x: cosine_similarity(x, embedding_query('아니 땐 굴뚝에 연기 나랴?'))
)
df.sort_values(by='cosine_similarity', ascending=False)


,속담,embedding,cosine_similarity
4,공든 탑이 무너지랴,"[-0.01824951171875, -0.017059326171875, -0.017...",0.248281
20,콩 심은 데 콩 나고 팥 심은 데 팥 난다,"[-0.010040283203125, -0.007045745849609375, -0...",0.233744
1,가는 날이 장날이다,"[-0.006801605224609375, -0.00921630859375, -0....",0.213224
18,백지장도 맞들면 낫다,"[-0.026611328125, 0.01503753662109375, -0.0183...",0.207951
16,바늘 도둑이 소 도둑 된다,"[-0.002910614013671875, -0.01047515869140625, ...",0.203440
14,모로 가도 서울만 가면 된다,"[-0.0211944580078125, -0.0003752708435058594, ...",0.202889
3,고래 싸움에 새우 등 터진다,"[0.0036220550537109375, -0.0070648193359375, -...",0.188418
11,뛰는 놈 위에 나는 놈 있다,"[0.0003631114959716797, -0.00197601318359375, ...",0.187549
13,매도 먼저 맞는 게 낫다,"[-0.01380157470703125, 0.003704071044921875, 0...",0.184172
2,개구리 올챙이 적 생각 못 한다,"[-0.00926971435546875, 0.0004954338073730469, ...",0.183821


In [25]:
df['cosine_similarity'] = df['embedding'].apply(
    lambda x: cosine_similarity(x, embedding_query('천 리 길도 한 걸음부터'))
)
df.sort_values(by='cosine_similarity', ascending=False)

,속담,embedding,cosine_similarity
22,시작이 반이다,"[0.00650787353515625, 0.0211029052734375, -0.0...",0.251237
12,말 한마디로 천 냥 빚 갚는다,"[-0.006191253662109375, 0.0087432861328125, 0....",0.243310
21,티끌 모아 태산,"[0.005161285400390625, 0.01270294189453125, -0...",0.222234
14,모로 가도 서울만 가면 된다,"[-0.0211944580078125, -0.0003752708435058594, ...",0.214886
5,구슬이 서 말이라도 꿰어야 보배,"[-0.0124053955078125, 0.0128173828125, -0.0065...",0.196937
16,바늘 도둑이 소 도둑 된다,"[-0.002910614013671875, -0.01047515869140625, ...",0.174723
18,백지장도 맞들면 낫다,"[-0.026611328125, 0.01503753662109375, -0.0183...",0.174441
1,가는 날이 장날이다,"[-0.006801605224609375, -0.00921630859375, -0....",0.171586
13,매도 먼저 맞는 게 낫다,"[-0.01380157470703125, 0.003704071044921875, 0...",0.170184
7,김칫국부터 마신다,"[-0.0004630088806152344, -0.01076507568359375,...",0.168761


In [26]:
df['cosine_similarity'] = df['embedding'].apply(
    lambda x: cosine_similarity(x, embedding_query('이왕이면 다홍 치마'))
)
df.sort_values(by='cosine_similarity', ascending=False)

,속담,embedding,cosine_similarity
19,보기 좋은 떡이 먹기도 좋다,"[-0.0095062255859375, 0.01593017578125, -0.005...",0.223448
13,매도 먼저 맞는 게 낫다,"[-0.01380157470703125, 0.003704071044921875, 0...",0.206585
0,가는 말이 고와야 오는 말이 곱다,"[-0.00739288330078125, 0.009613037109375, -0.0...",0.193449
17,배보다 배꼽이 크다,"[0.0152587890625, -0.01018524169921875, -0.005...",0.186029
14,모로 가도 서울만 가면 된다,"[-0.0211944580078125, -0.0003752708435058594, ...",0.181494
1,가는 날이 장날이다,"[-0.006801605224609375, -0.00921630859375, -0....",0.178873
5,구슬이 서 말이라도 꿰어야 보배,"[-0.0124053955078125, 0.0128173828125, -0.0065...",0.163007
12,말 한마디로 천 냥 빚 갚는다,"[-0.006191253662109375, 0.0087432861328125, 0....",0.160752
7,김칫국부터 마신다,"[-0.0004630088806152344, -0.01076507568359375,...",0.160438
18,백지장도 맞들면 낫다,"[-0.026611328125, 0.01503753662109375, -0.0183...",0.157129
